In [1]:
import pandas as pd

In [8]:
df = pd.read_excel('Preppin Data Challenge.xlsx')
df[['Name', 'Age', 'Area of Work']] = df['Name, Age, Area of Work'].str.extract('(\w+)\,\s(\w+)\:\s(.*)')
df.drop(columns='Name, Age, Area of Work', inplace=True)
df = df[~df['Name'].isna()]
df

,Project,2021-02-01 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,Name,Age,Area of Work
0,Client Meetings,NaN,2.0,NaN,1.0,NaN,1.5,0.5,NaN,NaN,Annual Leave,Dan,28,Client
1,Client Issues,1.0,1.5,4.5,3.5,1.00,2.0,1.0,2.0,3.0,Annual Leave,Dan,28,Client
2,Monthly Reports,NaN,NaN,NaN,NaN,2.00,1.0,1.0,2.0,1.0,Annual Leave,Dan,28,Client
3,Client Emails,2.0,0.5,0.5,0.5,1.00,1.0,1.0,NaN,NaN,Annual Leave,Dan,28,Client
4,Client Communications,1.0,1.0,NaN,NaN,NaN,0.5,NaN,NaN,NaN,Annual Leave,Dan,28,Client
5,Virtual Space,NaN,NaN,NaN,0.5,1.00,1.5,0.5,1.5,NaN,Annual Leave,Dan,28,Special Projects
6,Presentation for Exec,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,1.0,Annual Leave,Dan,28,Special Projects
7,Webinar,NaN,NaN,NaN,0.5,NaN,NaN,0.5,NaN,NaN,Annual Leave,Dan,28,Special Projects
8,Grad Scheme Organisation,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Annual Leave,Dan,28,Special Projects
9,Team Social,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,Annual Leave,Dan,28,Special Projects


In [31]:
df_melted = df.melt(id_vars=['Name', 'Age', 'Area of Work', 'Project'], var_name='Date', value_name='Hours')
df_melted.dropna(inplace=True)
df_melted = df_melted.loc[df_melted['Hours'] != 'Annual Leave'].copy()
df_melted['Hours'] = df_melted['Hours'].astype('float')
df_melted

,Name,Age,Area of Work,Project,Date,Hours
1,Dan,28,Client,Client Issues,2021-02-01,1.0
3,Dan,28,Client,Client Emails,2021-02-01,2.0
4,Dan,28,Client,Client Communications,2021-02-01,1.0
8,Dan,28,Special Projects,Grad Scheme Organisation,2021-02-01,1.0
12,Dan,28,Chats,Team Meetings,2021-02-01,2.0
...,...,...,...,...,...,...
494,Sam,45,Client,Client Issues,2021-02-12,2.0
496,Sam,45,Client,Client Emails,2021-02-12,1.5
497,Sam,45,Client,Client Communications,2021-02-12,1.0
503,Sam,45,Special Projects,Social Committee,2021-02-12,1.0


In [42]:
df_tot_hours_per_day = df_melted.groupby('Name', as_index=False).agg({'Hours':'sum', 'Date':'nunique'})
df_tot_hours_per_day['Average Hours'] = df_tot_hours_per_day['Hours']/df_tot_hours_per_day['Date']
df_tot_hours_per_day.drop(columns=['Date', 'Hours'], inplace=True)
df_tot_hours_per_day

,Name,Average Hours
0,Dan,8.027778
1,George,8.400000
2,Sam,7.700000


In [48]:
df_melted_WO_chats = df_melted.loc[df_melted['Area of Work'] != 'Chats'].copy()
df_tot_hours_per_area = df_melted_WO_chats.groupby(['Name', 'Area of Work'], as_index=False)['Hours'].sum()
df_tot_hours_per_area['Tot Hours'] = df_tot_hours_per_area.groupby('Name')['Hours'].transform('sum')
df_tot_hours_per_area['%'] = df_tot_hours_per_area['Hours']/df_tot_hours_per_area['Tot Hours'] * 100
df_tot_hours_per_area = df_tot_hours_per_area.loc[df_tot_hours_per_area['Area of Work'] == 'Client'].copy()
df_tot_hours_per_area.drop(columns=['Area of Work', 'Hours', 'Tot Hours'], inplace=True)
df_tot_hours_per_area

,Name,%
0,Dan,75.000000
2,George,81.294964
4,Sam,86.885246


In [50]:
df_output = pd.merge(df_tot_hours_per_day, df_tot_hours_per_area, on='Name')
df_output

,Name,Average Hours,%
0,Dan,8.027778,75.000000
1,George,8.400000,81.294964
2,Sam,7.700000,86.885246
